In [1]:
import os
import pandas as pd
import tensorflow as tf

2024-04-25 12:35:28.242896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 12:35:28.242989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 12:35:28.422252: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading model 


In [ ]:
!pip install /kaggle/input/acceleratez/accelerate-0.28.0-py3-none-any.whl
!pip install /kaggle/input/bitsandbytez/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl

In [ ]:
import torch

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3", quantization_config=bnb_config)

# Test model

In [ ]:
input_text = "Write a poem about a cat."
input_ids = tokenizer(input_text, return_tensors='pt').to("cuda")

res = model.generate(**input_ids, max_length=512)
print(tokenizer.decode(res[0]))

# Load Datasets


In [ ]:
# https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize/data

# Load the training data
train_data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')

# Load the test data
test_data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')

# Now you can view the first few rows of each to confirm they're loaded correctly
olympiad_data = pd.concat([train_data, test_data], ignore_index=True)



In [ ]:
# Clean data
olympiad_data.dropna()
#View 
olympiad_data.head()


In [ ]:
#clean this 
# https://huggingface.co/datasets/hendrycks/competition_math
from datasets import load_dataset

# Load the Hugging Face dataset
MATH_ds = load_dataset("hendrycks/competition_math")
train_dataset = MATH_ds['train']
test_dataset = MATH_ds['test']

# Convert to Pandas DataFrames
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()

# Concatenate the train and test dataframes
MATH_data = pd.concat([train_df, test_df], ignore_index=True)



In [ ]:
unique_links = MATH_data['type'].unique()
print(unique_links)
for link in unique_links:
    print(link)

In [ ]:
# No need to do regex
aime_data = pd.read_csv('/kaggle/input/aime-problem-set-1983-2024/AIME_Dataset_1983_2024.csv')

In [ ]:
#Fix structure of columns
# rm Year, Problem Number, Part
aime_data = aime_data.drop(['Year', 'Problem Number', 'Part'], axis=1)
aime_data.dropna()
#rename Question: question, Answer:answer, ID:id
aime_data.rename(columns={'ID': 'id','Question': 'problem', 'Answer': 'answer'}, inplace=True)
#View
aime_data.head()


In [ ]:
#clean this
amio_24_data = pd.read_csv('/kaggle/input/aimo-24-data-art-of-problem-solving/problems.csv')


In [ ]:
#View all the different values of the only the links 
#take out unneeded data after each cell run 
# List of patterns to remove rows for
patterns_to_remove = ['AHSME', 'AJHSME', 'USOMO', 'USAMO', 'USAJMO', 'USOJMO', 'AMC_10', 'AMC_8']

# Create a boolean mask where True indicates that a row should be deleted
mask = amio_24_data['link'].str.contains('|'.join(patterns_to_remove))

# Invert the mask to keep rows that do not contain any of the patterns
amio_24_data = amio_24_data[~mask]

# Now, 'amio_24_data' contains only the rows where 'link' doesn't include the specified patterns
unique_links = amio_24_data['link'].unique()

In [ ]:
#Fix structure of columns
#rm link, no
amio_24_data = amio_24_data.drop(['link', 'no'], axis=1)
#drop na
amio_24_data.dropna()
#View
amio_24_data.head()


In [ ]:
# clean this
amio_data = pd.read_csv('/kaggle/input/amio-parsed-art-of-problem-solving-website/parsed_ArtOfProblemSolving.csv')
amio_data.head()

patt_to_remove = ['AHSME', 'AJHSME', 'USOMO', 'USAMO', 'USAJMO', 'USOJMO', 'AMC_10', 'AMC_8']


# Create a boolean mask where True indicates that a row should be deleted
mask = amio_data['link'].str.contains('|'.join(patt_to_remove))

# Invert the mask to keep rows that do not contain any of the patterns
amio_data = amio_data[~mask]

# Now, 'amio_24_data' contains only the rows where 'link' doesn't include the specified patterns
unique_links = amio_data['link'].unique()
print(unique_links)

In [ ]:
#Fix structure of columns
#rm link, letter
amio_data = amio_data.drop(['link', 'letter'], axis=1)
#change problem_id to id
amio_data.rename(columns={'problem_id': 'id'}, inplace=True)
#drop na
amio_data.dropna()
#View
amio_data.head()

# Concatenate datsets 

In [ ]:
# For troubleshooting
# olympiad_data aime_data amio_24_data amio_data
print("Number of rows in olympiad_data:", len(olympiad_data))
print("Number of rows in aime_data:", len(aime_data))
print("Number of rows in amio_24_data:", len(amio_24_data))
print("Number of rows in amio_data:", len(amio_data))

In [ ]:
# All data variables: olympiad_data MATH_data aime_data amio_24_data amio_data
# Exlude MATH because can't get specific AIME/AMC10-12
data = pd.concat([olympiad_data, aime_data], ignore_index=True)
# Isolate analysis of amio_24_data and amio_data due to the solution column
solution_data = pd.concat([amio_data, amio_24_data], ignore_index=True)

#Drop duplicates
#!!!Though, different id's so not sure if the duplicates remove
data = data.drop_duplicates(subset=['problem'], keep='first')
solution_data = solution_data.drop_duplicates(subset=['problem'], keep='first')

print("Number of rows in solution_data:", len(solution_data))
print("Number of rows in data:", len(data))

# Maybe just concat the datasets


In [ ]:
combined_data = pd.concat([olympiad_data, aime_data, amio_data, amio_24_data], ignore_index=True)
print(f'Length beofre cleaning: {len(combined_data)}')
# Prioritize the rows that have 'solution' filled out 
combined_data_sorted = combined_data.sort_values(by='solution', ascending=False, na_position='last')
# Drop duplicates
df = combined_data_sorted.drop_duplicates(subset=['problem'], keep='first')
print(f'Length beofre cleaning{len(df)}')

In [ ]:
df.head()

# Preprocessing data

# Model Training

# Final prediction

In [ ]:
print(X.shape)  # This should match the first dimension of Y
print(Y.shape)

In [ ]:

# Assuming 'model' is your previously trained model
predictions = model.predict(X)

# If the model outputs probabilities (typical for classification tasks), and you need to convert these to class labels:
predictions = np.argmax(predictions, axis=1)  # Use this if your model is a multi-class classifier
# predictions = (predictions > 0.5).astype(int)  # Use this for binary classifiers, if predictions are probabilities

# Calculate accuracy
accuracy = accuracy_score(Y, predictions)
print("Accuracy:", accuracy)

In [ ]:
# Save the submission file
predictions.to_csv('submission.csv', index=False)
